In [28]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
import ipywidgets as widgets
from ipywidgets import interact

def plot_binomial_and_power(n=40, p0=0.5, p1=0.6, alpha=0.05, spiegelung=False):
    k = np.arange(0, n + 1)
    
    # Wahrscheinlichkeitsverteilungen berechnen
    binom_probs_p0 = binom.pmf(k, n, p0)
    binom_probs_p1 = binom.pmf(k, n, p1)
    
    # Kritischen Wert bestimmen
    cumulative_probs_p0 = 1 - binom.cdf(k - 1, n, p0)
    k_min = np.min(k[cumulative_probs_p0 <= alpha])
    
    # Power-Funktion g(p)
    p_values = np.linspace(0, 1, 500)
    power_values = [1 - binom.cdf(k_min - 1, n, p) for p in p_values]
    
    # Kritischen Bereich für den rechtsseitigen Test bestimmen
    critical = [i for i in k if i >= k_min]
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8))
    
    title_str1 = (
        fr"$H_0: p_0={p0}; H_1: p={p1}; (n={n}, \alpha={alpha}, k_{{\min}}={k_min})$"
    )
    title_str2 = (
        fr"$g(p) = P_p(X \geq {k_min}); n={n}; k_{{\min}}={k_min}; p_0={p0}; p_1={p1}$"
    )

    # Plot 1: Binomialverteilungen
    ax1.bar(k, binom_probs_p0, width=0.5, 
            color=['tomato' if i in critical else 'skyblue' for i in k], 
            label=f"$H_0$: p = {p0}")
    ax1.axhline(y=0, color='black', linewidth=1)  # x-Achse hervorheben
    if spiegelung:
        ax1.bar(k, -binom_probs_p1, width=0.5, 
                color=['lightgreen' if i in critical else 'tomato' for i in k], 
                label=f"$H_1$: p = {p1} (gespiegelt)")
    else:
        ax1.bar(k, binom_probs_p1, width=0.5, 
                color=['lightgreen' if i in critical else 'tomato' for i in k], 
                label=f"$H_1$: p = {p1}")
    
    ax1.axvspan(k_min - 0.5, n + 0.5, color='gray', alpha=0.2, label="Ablehnungsbereich")
    
    ax1.set_xlabel(r"$k$")
    ax1.set_ylabel(r"$P(X = k)$" + (" (H1 gespiegelt)" if spiegelung else ""))
    ax1.set_title(title_str1)
    ax1.legend()
    ax1.grid(True, linestyle='--', alpha=0.7)
    ax1.set_ylim(-max(max(binom_probs_p0), max(binom_probs_p1)) * 1.3,
                  max(max(binom_probs_p0), max(binom_probs_p1)) * 1.3)
    ax1.set_xlim(-1, n + 1)
    
    # Plot 2: Power-Funktion g(p)
    ax2.plot(p_values, power_values, color='black', linewidth=2, label=r"$g(p)$")
    ax2.plot([p0, p0], [0, 1], color='gray', linestyle='--')
   
    # Fehler 2. Art β markieren und in die Mitte setzen
    power_val = binom.sf(k_min-1, n, p1)
    ax2.scatter([p1], [power_val], color='black', zorder=4)
    ax2.text(p1 + 0.01, power_val/2, fr"$1-\beta = {power_val:.3f}$", 
             verticalalignment='bottom', fontsize=12, color="green")
    ax2.text(p1 + 0.01, power_val/2+0.5, fr"$\beta = {1-power_val:.3f}$", 
             verticalalignment='bottom', fontsize=12, color="red")
    
    # Power des Tests markieren (1-β)
    ax2.plot([p1, p1], [0, power_val], color='green', linestyle='--')
    ax2.plot([p1, p1], [power_val,1], color='red', linestyle='--')
    
    ax2.set_xlabel(r"$p$")
    ax2.set_ylabel(r"$P(X \geq k_{\min})$")
    ax2.set_title(title_str2)
    ax2.legend()
    ax2.grid(True, linestyle='--', alpha=0.7)
    ax2.set_ylim(0, 1.1)
    ax2.set_xlim(0, 1)
    
    plt.tight_layout()
    plt.show()

# Interaktive Darstellung mit Slidern und Kontrollschalter für Spiegelung
interact(
    plot_binomial_and_power,
    n=widgets.IntSlider(min=10, max=200, step=10, value=40, description='n'),
    #p0=widgets.FloatSlider(min=0.1, max=0.9, step=0.01, value=0.5, description='p₀'),
     p0=widgets.FloatSlider(min=0.1, max=0.9, step=0.01, value=0.5, description='p0'),
    #p1=widgets.FloatSlider(min=0.1, max=0.9, step=0.01, value=0.6, description='p₁'),
    p1=widgets.FloatSlider(min=0.1, max=0.9, step=0.01, value=0.6, description='p1'),
    #alpha=widgets.FloatSlider(min=0.01, max=0.2, step=0.01, value=0.05, description='α'),
    alpha=widgets.FloatSlider(min=0.01, max=0.2, step=0.01, value=0.05, description='alpha'),
    spiegelung=widgets.Checkbox(value=True, description="Spiegelung aktivieren")
)
print()

interactive(children=(IntSlider(value=40, description='n', max=200, min=10, step=10), FloatSlider(value=0.5, d…